# Image summary and visual question answering

This notebooks shows how to generate image captions and use the visual question answering with [LAVIS](https://github.com/salesforce/LAVIS) library. 

The first cell is only run on google colab and installs the [ammico](https://github.com/ssciwr/AMMICO) package.

After that, you can import `ammico` and read in the files given a folder path.

In [ ]:
# if running on google colab
# flake8-noqa-cell
import os

if "google.colab" in str(get_ipython()):
    # update python version
    # install setuptools
    # %pip install setuptools==61 -qqq
    # uninstall pre-installed 'tensorflow-probability' due to incompatibility
    %pip uninstall tensorflow-probability dopamine-rl lida pandas-gbq torchaudio torchdata torchtext orbax-checkpoint -y -qqq
    # install ammico
    %pip install git+https://github.com/ssciwr/ammico.git -qqq
    # mount google drive for data and API key
    from google.colab import drive

    drive.mount("/content/drive")

In [ ]:
import ammico

In [ ]:
# Here you need to provide the path to your google drive folder
# or local folder containing the images
mydict = ammico.find_files(
    path="/content/drive/MyDrive/misinformation-data/",
    #path="../../data/images/",
    limit=2,
)
mydict

## Create captions for images and directly write to csv

Here you can choose between models: `"base"` or `"large"`. This will generate the caption for each image and directly put the results in your dictionary `mydict`. Then you can transform it into the dataframe and this dataframe can be exported as a .csv file.

The results are written in the columns: 
- `const_image_summary` - the permanent summaries, which do not change from run to run (analyse_image).
- `3_non-deterministic_summary` displays three different summaries generated with different seeds that change from run to run (analyse_image). 

You can also specify what kind of analysis you want to perform with `analysis_type`. `"summary"` will generate a summary for all pictures in your dictionary `mydict`, `"questions"` will prepare answers to your questions for all pictures, and `"summary_and_questions"` will do both. 
If you load the models (`summary_model`, `summary_vis_processors` for `"summary"` and `summary_vqa_model`, `summary_vqa_vis_processors`, `summary_vqa_txt_processors` for `"questions"`) into memory beforehand and pass them to the function, it can speed up the analysis many times. 



In [ ]:
obj = ammico.SummaryDetector(mydict, analysis_type="summary", model_type="base")    # here we load the base model to the memory. This can dramatically speed up the calculation process then.
#obj = ammico.SummaryDetector(mydict, analysis_type="summary", model_type="large")

In [ ]:
for key in mydict:
    mydict[key] = obj.analyse_image(subdict = mydict[key], analysis_type="summary")

In [ ]:
mydict

### Convert to dataframe and write csv

Convert the dictionary of dictionarys into a dictionary with lists:

In [ ]:
df = ammico.get_dataframe(mydict)

Check the dataframe:

In [ ]:
df.head(10)

Write the csv file:

In [ ]:
df.to_csv("/content/drive/MyDrive/misinformation-data/data_out.csv")

## Generate answers to free-form questions about images written in natural language. 

Load the model to the memory through object creation with parameters `analysis_type="questions"` and `model_type="vqa"`.  Set the list of questions as a list of strings `list_of_questions`,   and pass them to the function

In [ ]:
list_of_questions = [
    "How many persons on the picture?",
    "Are there any politicians in the picture?",
    "Does the picture show something from medicine?",
]

In [ ]:
obj = ammico.SummaryDetector(mydict, analysis_type="questions",  model_type="vqa")

In [ ]:
for key in mydict:
    mydict[key] = obj.analyse_image(subdict = mydict[key], analysis_type="questions", list_of_questions = list_of_questions)

In [ ]:
mydict

Or you can perform two types of analysis at a time `analysis_type="summary_and_questions"`.

In [ ]:
obj = ammico.SummaryDetector(mydict, analysis_type="summary_and_questions", model_type="base", list_of_questions = list_of_questions)

In [ ]:
for key in mydict:
    mydict[key] = obj.analyse_image(subdict = mydict[key], analysis_type="summary_and_questions", list_of_questions = list_of_questions)

### Convert to dataframe and write csv
These steps are required to convert the dictionary of dictionarys into a dictionary with lists, that can be converted into a pandas dataframe and exported to a csv file.

In [ ]:
df2 = ammico.get_dataframe(mydict)

In [ ]:
df2.head(10)

In [ ]:
df2.to_csv("/content/drive/MyDrive/misinformation-data/data_out2.csv")

## Manually inspect the summaries and visual question answering

To check the analysis, you can inspect the analyzed elements here. Loading the results takes a moment since it loads the big model to memory for every picture, so please be patient. If you are sure of what you are doing. In this widget you can select the picture, the type of analysis and the question.


In [ ]:
analysis_explorer = ammico.AnalysisExplorer(mydict)
analysis_explorer.run_server(port=8055)

# New models
This is very heavy models. They requare approx 60GB of RAM and they can use 20+GB memory GPUs for acceleration.

In [ ]:
obj = ammico.SummaryDetector(mydict, analysis_type = "summary_and_questions", model_type = "blip2_t5_caption_coco_flant5xl", device_type= "cpu")
# list of the new models that can be used:
# "blip2_t5_pretrain_flant5xxl",
# "blip2_t5_pretrain_flant5xl",
# "blip2_t5_caption_coco_flant5xl",
# "blip2_opt_pretrain_opt2.7b",
# "blip2_opt_pretrain_opt6.7b",
# "blip2_opt_caption_coco_opt2.7b",
# "blip2_opt_caption_coco_opt6.7b",

# You can use `pretrain_` model types for zero-shot image-to-text generation with prompts.
# Or you can use `caption_coco_`` model types to generate coco-style captions.
# `flant5` and `opt` means that the model equipped with FlanT5 and OPT LLMs respectively.

#also you can perform all calculation on cpu if you set device_type= "cpu" or gpu if you set device_type= "cuda"

In [ ]:
for key in mydict:
    mydict[key] = obj.analyse_image(subdict = mydict[key], analysis_type="summary_and_questions")

# analysis_type can be 
# "summary",
# "questions",
# "summary_and_questions".

In [ ]:
mydict

You can also pass a list of questions to this cell if `analysis_type="summary_and_questions"` or `analysis_type="questions"`. But the format of questions has changed in new models. 

Here is an example of a list of questions:

In [ ]:
list_of_questions = [
    "Question: Are there people in the image? Answer:",
    "Question: What is this picture about? Answer:",
]

In [ ]:
for key in mydict:
    mydict[key] = obj.analyse_image(subdict = mydict[key], analysis_type="questions", list_of_questions=list_of_questions)

You can also pass a question with previous answers as context into this model and pass in questions like this one to get a more accurate answer:

You can combine as many questions as you want in a single query as a list.

In [ ]:
list_of_questions = [
    "Question: What country is in the picture? Answer: USA. Question: Why? Answer: Because there is an American flag in the background . Question: Where it comes from? Answer:",
    "Question: Which city is this? Answer: Frankfurt. Question: why?",
]

In [ ]:
for key in mydict:
    mydict[key] = obj.analyse_image(subdict = mydict[key], analysis_type="questions", list_of_questions=list_of_questions)

In [ ]:
mydict

You can also ask sequential questions if you pass the argument `cosequential_questions=True`. This means that the answers to previous questions will be passed as context to the next question. However, this method will work a bit slower, because for each image the answers to the questions will not be calculated simultaneously, but sequentially. 

In [ ]:
list_of_questions = [
    "Question: Is this picture taken inside or outside? Answer:",
    "Question: Why? Answer:",
]

In [ ]:
for key in mydict:
    mydict[key] = obj.analyse_image(subdict = mydict[key], analysis_type="questions", list_of_questions=list_of_questions, consequential_questions=True)

In [ ]:
mydict

### Convert to dataframe and write csv

Convert the dictionary of dictionarys into a dictionary with lists:

In [ ]:
df = ammico.get_dataframe(mydict)

Check the dataframe:

In [ ]:
df.head(10)

Write the csv file:

In [ ]:
df.to_csv("/content/drive/MyDrive/misinformation-data/data_out.csv")